In [5]:
from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np

In [35]:
def dataSetAudios(nombre):
    lista = []
    listafin = []
    for a in range(1,4):
        audio = AudioSegment.from_file(f'../audios/{nombre}{a}.m4a', format='mp4')
        lista.append(audio)
    for a in lista:
        array = a.get_array_of_samples()
        four = fft(array)
        listafin.append(four)
    
    return listafin        

In [38]:
af = "Afecto"
ale = "Alegria"
enf = "Enfado"
trs = "Tristeza"
afecto = dataSetAudios(af)
alegria = dataSetAudios(ale)
enfado = dataSetAudios(enf)
tristeza = dataSetAudios(trs)